# Route Identification Over Time, Approach 2
See `06_route_identification_over_time.ipynb` for previous work.
<br>Determined that more data is needed than what is given from `fct_monthly_routes`. exploring data from `helpers.import_scheduled_trips` to get feed_key, route short/long name/id/desc and other. 
<br>Cannot rely on feed_key as it is unstable over time. Next best is gtfs_schedule key, but absolute best is source_record_id.
<br>Start with running `make install_env` to install everything needed. `helpers`,`rt_dates`,`sched_gcs`
<br>Query data from warehouse, then use this snippet from `make_routes_gdf` from `_shared_utils/shared_utils/geography_utils.py`.
* ddf["geometry"] = ddf.pt_array.apply(make_linestring)
    
<br> then save out as geo parquet to the `gtfs_schedule` folder in GCS (so versioning and history stays) using 
* utils.geoparquet_gcs_export(
                vp_gdf,
                SEGMENT_GCS,
                f"vp_{analysis_date}"
            )

<br>also add `route_long_name` to query. then see if we are able to see any changes in routes via:
* feed key
* name
* route id
* route short name
* route long name
* route desc
    

In [1]:
import pandas as pd
#rt_segment_speeds/segment_speed_utils
from segment_speed_utils import helpers

#rt_segment_speeds/segment_speed_utils/project_vars.py
from segment_speed_utils.project_vars import SCHED_GCS

#_shared_utils/shared_utils/rt_dates.py
from shared_utils import rt_dates



## 11/15/2023

Latest update. Attempting to join `fct_monthly_routes` & `dim_providers_gtfs_data`. then will joing in data from this approach

In [ ]:
#DO NOT RUN



# grabbing additional dates
# april_date = rt_dates.DATES["apr2023"]
# may_date = rt_dates.DATES["may2023"]
# june_date = rt_dates.DATES["jun2023"]
# july_date = rt_dates.DATES["jul2023"]
# aug_date = rt_dates.DATES["aug2023"]
# sep_date = rt_dates.DATES["sep2023"]

In [ ]:
#DO NOT RUN
# list of all the months
# months_list = [sep_date, aug_date, april_date, may_date, june_date, july_date]

In [ ]:
#DO NOT RUN
# test loop of import_scheduled_trips, iterated through months_list

# empty list
# month_trips = {}

# for months in months_list:
#     df = helpers.import_scheduled_trips(
#         months,
#         columns=[
#             "feed_key",
#             "name",
#             "gtfs_dataset_key",
#             "route_id",
#             "route_short_name",
#             "route_long_name",
#             "route_desc",
#         ],
#         filters=[[("name", "==", "Sacramento Schedule")]],
#         get_pandas=True,
#     )
#     month_trips[months] = df

In [ ]:
#  month_trips.keys()

In [ ]:
# april = month_trips['2023-04-12'].assign(month='april')
# may = month_trips['2023-05-17'].assign(month='may')
# june = month_trips['2023-06-14'].assign(month='june')
# july = month_trips['2023-07-12'].assign(month='july')
# aug = month_trips['2023-08-15'].assign(month='aug')
# sep = month_trips['2023-09-13'].assign(month='sep')

In [ ]:
# use this pseudo code to concat all the different months data
# pd.concat can stack DFs on top of others.
# pd.concat([df1, df2], axis=0)

In [ ]:
#DO NOT RUN
#test to see if i can use a dictionary in concat
# all_trips = pd.concat([april,may,june,july,aug,sep],axis=0)

In [ ]:
#DO NOT RUN
#saving dataframe to gcs
#TEST CODE aprl_sept_2023_routes.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/route_identification_2023_m04_m09.parquet')

# all_trips.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_trips_route_identification_2023_m04_m09.parquet')

In [2]:
#from now on, READ IN DATA FROM GCS

sac_trips = pd.read_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_trips_route_identification_2023_m04_m09.parquet')

In [3]:
display(sac_trips.shape)
display(type(sac_trips))
display(sac_trips.head())

(326, 8)

pandas.core.frame.DataFrame

,feed_key,name,schedule_gtfs_dataset_key,route_id,route_short_name,route_long_name,route_desc,month
0,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,001,1,GREENBACK,None,april
1,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,011,11,NATOMAS/LAND PARK,None,april
2,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,013,13,NATOMAS/ARDEN,None,april
3,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,015,15,DEL PASO HEIGHTS,None,april
4,55c36112e9a6a849c0ddc65eb54de7f1,Sacramento Schedule,43a1e46d592a1ee647bce8422c68460c,019,19,RIO LINDA,None,april


In [5]:
#No data for June 2023
sac_trips.month.value_counts()

sep      66
april    65
may      65
july     65
aug      65
Name: month, dtype: int64

In [6]:
# 2 unique gtfs dataset keys, as opposed to 3 unique feed keys
sac_trips.schedule_gtfs_dataset_key.value_counts()

43a1e46d592a1ee647bce8422c68460c    260
cb3074eb8b423dfc5acfeeb0de95eb82     66
Name: schedule_gtfs_dataset_key, dtype: int64

In [21]:
#which months had which schedule dataset key?
sac_trips.pivot_table(index = ['schedule_gtfs_dataset_key','month'],
                      values = ['feed_key'],
                     aggfunc='count'
                             ).reset_index()

,schedule_gtfs_dataset_key,month,feed_key
0,43a1e46d592a1ee647bce8422c68460c,april,65
1,43a1e46d592a1ee647bce8422c68460c,aug,65
2,43a1e46d592a1ee647bce8422c68460c,july,65
3,43a1e46d592a1ee647bce8422c68460c,may,65
4,cb3074eb8b423dfc5acfeeb0de95eb82,sep,66


In [ ]:
#expect to see 5 instances of each route id (1 for each month, except june). but some routes only happen 1 time? route 10 and 137
sac_trips.route_id.value_counts()

In [ ]:
#similar results with route short name
sac_trips.route_short_name.value_counts()

In [ ]:
#further investigation of route 10 and 137, see what their rows say
#only 1 row for each of these routes, both occuring in september
display(sac_trips[sac_trips['route_id']=='10'])
display(sac_trips[sac_trips['route_id']=='137'])

In [ ]:
#other routes return multiple rows as expected. 
sac_trips[sac_trips['route_id']=='F10']

In [ ]:
sac_trips.sort_values(by=['route_id', 'month'])

In [ ]:
#would like shape Id and pt array from previous approach, but need a join to make this work
#seek out preliminary joins via metabase
